In [1]:
! pip install pandas 

In [2]:
! pip install bs4
! pip install requests
! pip install selenium

In [3]:
! pip install selenium


In [7]:
import re
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time  # To allow for delays while loading new pages

# Set up Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Load the main page
url = 'http://www.tunisie-annonce.com/AnnoncesImmobilier.asp'
driver.get(url)
print("Page title:", driver.title)

# Parse the main page
page = soup(driver.page_source, 'html.parser')

# Extract property ads
ads = page.find_all("tr", class_="Tableau1")  # Rows containing ads
print(f"Number of ads found: {len(ads)}")

# Function to extract number of rooms
def extract_rooms(category, description):
    category_match = re.search(r'App\.\s*(\d+)', category)
    if category_match:
        return int(category_match.group(1)) - 1  # Convention (e.g., 3 pièces = 2 rooms)

    description_match = re.search(r'\bS(\d+)\b', description, re.IGNORECASE)
    if description_match:
        return int(description_match.group(1))
    return "N/A"

# Function to extract surface area from detailed page
def extract_surface(details_url):
    try:
        driver.get(details_url)  # Load the detailed page
        time.sleep(2)  # Allow time for the page to load
        details_page = soup(driver.page_source, 'html.parser')

        # Find the "Surface" field
        surface_row = details_page.find("td", class_="da_label_field", string="Surface")
        if surface_row:
            surface = surface_row.find_next_sibling("td").text.strip()
            # Replace non-breaking space (\xa0) with a regular space
            surface_cleaned = surface.replace("\xa0", " ")
            return surface_cleaned
        return "N/A"
    except Exception as e:
        print(f"Error extracting surface: {e}")
        return "N/A"

# Extract and display ad details
data = []
base_url = "http://www.tunisie-annonce.com/"
for ad in ads:
    try:
        cols = ad.find_all("td")
        if len(cols) >= 12:
            location = cols[1].text.strip()
            type_house = cols[3].text.strip()
            category = cols[5].text.strip()
            description_tag = cols[7].find("a")
            description = description_tag.text.strip() if description_tag else "N/A"
            price = cols[9].text.strip()
            date = cols[11].text.strip()
            rooms = extract_rooms(category, description)

            # Extract link to detailed page
            details_link = base_url + description_tag["href"] if description_tag and "href" in description_tag.attrs else None
            surface = extract_surface(details_link) if details_link else "N/A"

            # Append data
            data.append({
                "Location": location,
                "Type": type_house,
                "Category": category,
                "Description": description,
                "Price": price,
                "Date": date,
                "Rooms": rooms,
                "Surface": surface
            })

    except Exception as e:
        print(f"Error parsing ad: {e}")

# Print extracted data
for entry in data:
    print(entry)

# Close the driver
driver.quit()


Page title: Petites Annonces Immobilier en Tunisie
Number of ads found: 25
{'Location': 'Cite El Bassati', 'Type': 'Terrain', 'Category': 'Terrain nu', 'Description': 'Adel  terrain khelija', 'Price': '66 000', 'Date': '18/12/2024', 'Rooms': 'N/A', 'Surface': '180 m²'}
{'Location': 'El Yasminette', 'Type': 'Vente', 'Category': 'Maisons', 'Description': 'Maison yasminette', 'Price': '180 000', 'Date': '18/12/2024', 'Rooms': 'N/A', 'Surface': '150 m²'}
{'Location': 'Bou Mhel', 'Type': 'Vente', 'Category': 'Duplex', 'Description': 'Maison  à boumhel', 'Price': '550 000', 'Date': '18/12/2024', 'Rooms': 'N/A', 'Surface': '156 m²'}
{'Location': 'Sidi Daoud', 'Type': 'Location', 'Category': 'Maisons', 'Description': 'Villa s4 avec jardin à l', 'Price': '4 000', 'Date': '18/12/2024', 'Rooms': 4, 'Surface': '1 m²'}
{'Location': 'Gammart', 'Type': 'Location', 'Category': 'Maisons', 'Description': 'Villa s3 meublée vue mer', 'Price': '9 500', 'Date': '18/12/2024', 'Rooms': 3, 'Surface': '1 m²'}
{